### Scraping tool to get reviews from restaurants. Firstly the function scrapeRestaurantsUrlsAll is used to gether all the restaurant urls based on location. Then we iterate through those urls and collect all the reviews.  

### Keyrðu cellurnar þangað til þú sérð

In [36]:
import requests 
from bs4 import BeautifulSoup
import csv 
from selenium import webdriver
import time
import sys
import os
import argparse
import string
import pickle

In [37]:
# Add urls for all "next" pages
def scrapeRestaurantsUrls(tripURLs):
    urls =[]
    for url in tripURLs:
        page = requests.get(url)
        soup = BeautifulSoup(page.text, 'html.parser')
        results = soup.find('div', class_='_1kXteagE')
        stores = results.find_all('div', class_='wQjYiB7z')
        for store in stores:
            unModifiedUrl = str(store.find('a', href=True)['href'])
            urls.append('https://www.tripadvisor.com'+unModifiedUrl)
    return urls

# Add urls for all "next" pages
def scrapeRestaurantsUrlsAll(url, limit=100):
    store_name = []
    urls = []
    limit_set = 1
    nextPage = True
    while nextPage and limit_set <= limit:
        page = requests.get(url)
        soup = BeautifulSoup(page.text, 'html.parser')
        results = soup.find('div', class_='_1kXteagE')
        stores = results.find_all('div', class_='wQjYiB7z') 
        for store in stores:
            if store.find('a', class_ = '_15_ydu6b').text[0].isdigit(): # skip the ones that er sponsored since they will also come later.
                
                print(store.find('a', class_ = '_15_ydu6b').text)
                unModifiedUrl = str(store.find('a', href=True)['href'])
                urls.append('https://www.tripadvisor.com'+unModifiedUrl)
        limit_set += 1
        #Go to next page if exists
        try:
            print('tried next in finding all')
            unModifiedUrl = str(soup.find('a', class_ = 'nav next rndBtn ui_button primary taLnk',href=True)['href'])
            # print(unModifiedUrl, 'later unmod')
            url = 'https://www.tripadvisor.com' + unModifiedUrl
            # print('new url is ', url)
        except:
            print('no next in finding all')
            nextPage = False

    with open(pathAllRestaurants, 'wb') as f:
        pickle.dump(urls, f)

    print(f'Total restaurant count: {len(urls)}')
    return urls

def scrapeRestaurantInfo(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    storeName = soup.find('h1', class_='_3a1XQ88S').text
    avgRating = soup.find('span', class_='r2Cf69qf').text.strip()
    storeAddress = soup.find('div', class_= '_2vbD36Hr _36TL14Jn').find('span', class_='_2saB_OSe').text.strip()
#     urlAddress = str(soup.find('div', class_ = '_2vbD36Hr _36TL14Jn').find('span').find('a', href=True)['href'])
    nrReviews = soup.find('a', class_='_10Iv7dOs').text.strip().split()[0]
    cousineType = [word.text for  word in soup.find('span', class_='_13OzAOXO _34GKdBMV').find_all('a')]
    nrPos = soup.find('a', class_='_15QfMZ2L').find('b').find('span').text.strip()
    with open(pathToStoreInfo, mode='a', encoding="utf-8") as trip:
        data_writer = csv.writer(trip, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
        data_writer.writerow([storeName, storeAddress, avgRating, nrReviews, cousineType[0], cousineType[1:], nrPos])

In [38]:
# driver = webdriver.Chrome('chromedriver.exe')
# This should be set to the path of the folder holding the python script.
    
def get_reviews(url):
    print(url)
    #if you want to scrape restaurants info

#     scrapeRestaurantInfo(url)

    nextPage = True
    while nextPage:
        #Requests
        driver.get(url)
        time.sleep(1)
        #Click More button
#         more = driver.find_elements_by_xpath("//span[contains(text(),'More')]")
        more = driver.find_elements_by_xpath("//span[@class='taLnk ulBlueLinks'][contains(text(),'More')]")
        # Push all buttons that unclude the "More" option on each review.
        for x in range(0,len(more)):
            try:
                driver.execute_script("arguments[0].click();", more[x])
                time.sleep(3)
            except:
                pass
            
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        #Store name
        storeName = soup.find('h1', class_='_3a1XQ88S').text
        #Reviews
        results = soup.find('div', class_='listContainer hide-more-mobile')
        try:
            reviews = results.find_all('div', class_='prw_rup prw_reviews_review_resp')
        except Exception:
            continue
        #Export to csv
        try:
            with open(pathToReviews, mode='a', encoding="utf-8") as trip_data:
                data_writer = csv.writer(trip_data, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
                for review in reviews:
                    ratingDate = review.find('span', class_='ratingDate').get('title')
                    reviewHeader = review.find('span', class_='noQuotes').text
                    text_review = review.find('p', class_='partial_entry')
                    if len(text_review.contents) > 2:
                        reviewText = str(text_review.contents[0][:-3]) + ' ' + str(text_review.contents[1].text)
                    else:
                        reviewText = text_review.text
                    reviewerUsername = review.find('div', class_='info_text pointer_cursor')
                    reviewerUsername = reviewerUsername.select('div > div')[0].get_text(strip=True)
                    rating = review.find('div', class_='ui_column is-9').findChildren('span')
                    rating = str(rating[0]).split('_')[3].split('0')[0]
                    data_writer.writerow([storeName, reviewerUsername, ratingDate, reviewHeader, reviewText, rating])
        except:
            pass

        #Go to next page if exists
        try:
            unModifiedUrl = str(soup.find('div', class_ = 'prw_rup prw_common_responsive_pagination').find('a', class_='nav next ui_button primary', href=True).get('href'))
            # unModifiedUrl = str(soup.find('a', class_ = 'nav next ui_button primary',href=True)['href'])
            url = 'https://www.tripadvisor.com' + unModifiedUrl
        except:
            nextPage = False


## INFO

Þú getur valið hversu mikið þú tekur í keyrslunni en ég held að 300 ætti alveg að vera fínt. 

Það er hægt að copy-a þessu notebook og keyra 2 samtímis þá breytiru bara 0:150 og 150:300 fyrir hvoru keyrsluna.

Ég sé svo um að eyða út fyrstu 300 línunum og skoða svo þau url sem voru með vesen "Save-a url með vesen". Þú mátt runna þá cellu einnig svo vandræðaslóðirnar séu vistaðar.

In [39]:
# Create new reviews.csv path for run
pathToReviews = "reviews9_kristin.csv"

with open(pathToReviews, mode='a', encoding="utf-8") as trip_data:
    data_writer = csv.writer(trip_data, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
    data_writer.writerow(['storeName', 'reviewerUsername', 'ratingDate', 'reviewHeader','reviewText', 'rating'])

In [40]:
# Þetta er aðalskjalið sem heldur utan um þá linka sem eru eftir.
with open("urls_left.txt", "rb") as f:   # Unpickling
    urls_left = pickle.load(f)

driver_path = f'{os.getcwd()}/chromedriver' # Get driver to run selenium
driver = webdriver.Chrome(driver_path)

finished = []
bad_url = []
rest_300 = urls_left[1500:2000]
i = 0
for url in rest_300:
    try:
        get_reviews(url)
        finished.append(url)
        print(i)
        i+=1
    except:
        bad_url.append(url)
        i+=1

https://www.tripadvisor.com/Restaurant_Review-g189541-d12949010-Reviews-Mat_99-Copenhagen_Zealand.html
0
https://www.tripadvisor.com/Restaurant_Review-g806262-d11814296-Reviews-Akiba_Sushi_Wok-Frederiksberg_Copenhagen_Zealand.html
1
https://www.tripadvisor.com/Restaurant_Review-g806262-d6428816-Reviews-Venus_Pizzeria-Frederiksberg_Copenhagen_Zealand.html
2
https://www.tripadvisor.com/Restaurant_Review-g189541-d21501280-Reviews-Popl-Copenhagen_Zealand.html
3
https://www.tripadvisor.com/Restaurant_Review-g806262-d15109990-Reviews-Okawari_Sushi-Frederiksberg_Copenhagen_Zealand.html
4
https://www.tripadvisor.com/Restaurant_Review-g189541-d15523604-Reviews-Guldgrillen-Copenhagen_Zealand.html
5
https://www.tripadvisor.com/Restaurant_Review-g189541-d11045691-Reviews-Brod_Vaerten-Copenhagen_Zealand.html
6
https://www.tripadvisor.com/Restaurant_Review-g806262-d11768645-Reviews-Thaiboksen-Frederiksberg_Copenhagen_Zealand.html
7
https://www.tripadvisor.com/Restaurant_Review-g189541-d896840-Review

71
https://www.tripadvisor.com/Restaurant_Review-g189541-d10642185-Reviews-HC_Andersens_Cafe-Copenhagen_Zealand.html
72
https://www.tripadvisor.com/Restaurant_Review-g806262-d5937033-Reviews-Cafe_CiCi-Frederiksberg_Copenhagen_Zealand.html
73
https://www.tripadvisor.com/Restaurant_Review-g189541-d15334997-Reviews-Yoburger-Copenhagen_Zealand.html
74
https://www.tripadvisor.com/Restaurant_Review-g189541-d15276683-Reviews-VACA_Trianglen-Copenhagen_Zealand.html
75
https://www.tripadvisor.com/Restaurant_Review-g189541-d9749057-Reviews-Gorm_s_Fisketorvet-Copenhagen_Zealand.html
76
https://www.tripadvisor.com/Restaurant_Review-g189541-d14942499-Reviews-Cafe_A-Copenhagen_Zealand.html
77
https://www.tripadvisor.com/Restaurant_Review-g806262-d15804273-Reviews-Spise_Bar15-Frederiksberg_Copenhagen_Zealand.html
78
https://www.tripadvisor.com/Restaurant_Review-g189541-d9453929-Reviews-Burgerklubben-Copenhagen_Zealand.html
79
https://www.tripadvisor.com/Restaurant_Review-g806262-d8089679-Reviews-Flint

143
https://www.tripadvisor.com/Restaurant_Review-g189541-d21343566-Reviews-Ugood-Copenhagen_Zealand.html
144
https://www.tripadvisor.com/Restaurant_Review-g189541-d21004512-Reviews-Dengronnekutter-Copenhagen_Zealand.html
145
https://www.tripadvisor.com/Restaurant_Review-g189541-d12543004-Reviews-Amager_Pizza_Palace-Copenhagen_Zealand.html
146
https://www.tripadvisor.com/Restaurant_Review-g189541-d14509359-Reviews-Izumi_Sushi-Copenhagen_Zealand.html
147
https://www.tripadvisor.com/Restaurant_Review-g189541-d11905298-Reviews-Sydvest_Bodega-Copenhagen_Zealand.html
148
https://www.tripadvisor.com/Restaurant_Review-g189541-d13416663-Reviews-Cafe_Garagen-Copenhagen_Zealand.html
149
https://www.tripadvisor.com/Restaurant_Review-g189541-d11938332-Reviews-Sydhavnens_Polsevogn-Copenhagen_Zealand.html
150
https://www.tripadvisor.com/Restaurant_Review-g189541-d17477961-Reviews-Sheddy_s_CPH-Copenhagen_Zealand.html
151
https://www.tripadvisor.com/Restaurant_Review-g189541-d14007613-Reviews-Lille_Is

216
https://www.tripadvisor.com/Restaurant_Review-g189541-d2357291-Reviews-Latitude_Cafe-Copenhagen_Zealand.html
217
https://www.tripadvisor.com/Restaurant_Review-g189541-d19863009-Reviews-ShaBaz_Kaffebar_Kokken-Copenhagen_Zealand.html
218
https://www.tripadvisor.com/Restaurant_Review-g189541-d17353980-Reviews-McDonald_s-Copenhagen_Zealand.html
219
https://www.tripadvisor.com/Restaurant_Review-g189541-d21302384-Reviews-Tommi_s_Burger_Joint_Norrebro-Copenhagen_Zealand.html
220
https://www.tripadvisor.com/Restaurant_Review-g189541-d12690842-Reviews-Ristobar-Copenhagen_Zealand.html
221
https://www.tripadvisor.com/Restaurant_Review-g189541-d23204545-Reviews-Domino_s_Pizza-Copenhagen_Zealand.html
222
https://www.tripadvisor.com/Restaurant_Review-g189541-d23204499-Reviews-Domino_s_Pizza-Copenhagen_Zealand.html
223
https://www.tripadvisor.com/Restaurant_Review-g189541-d21340770-Reviews-Nabo-Copenhagen_Zealand.html
224
https://www.tripadvisor.com/Restaurant_Review-g189541-d13975849-Reviews-Ala

289
https://www.tripadvisor.com/Restaurant_Review-g806262-d19084152-Reviews-Master_Kina_Grill-Frederiksberg_Copenhagen_Zealand.html
290
https://www.tripadvisor.com/Restaurant_Review-g806262-d19727325-Reviews-Social_Food-Frederiksberg_Copenhagen_Zealand.html
291
https://www.tripadvisor.com/Restaurant_Review-g806262-d20976210-Reviews-Meli_s-Frederiksberg_Copenhagen_Zealand.html
292
https://www.tripadvisor.com/Restaurant_Review-g806262-d13008534-Reviews-Joe_The_Juice-Frederiksberg_Copenhagen_Zealand.html
293
https://www.tripadvisor.com/Restaurant_Review-g806262-d12912916-Reviews-Coffee_Bagel_House-Frederiksberg_Copenhagen_Zealand.html
294
https://www.tripadvisor.com/Restaurant_Review-g806262-d18338049-Reviews-Evergreens-Frederiksberg_Copenhagen_Zealand.html
295
https://www.tripadvisor.com/Restaurant_Review-g806262-d18976389-Reviews-Floyd_s-Frederiksberg_Copenhagen_Zealand.html
296
https://www.tripadvisor.com/Restaurant_Review-g806262-d23200320-Reviews-Domino_s_Pizza-Frederiksberg_Copenhag

## Save-a url með vesen

In [41]:
# For curiosity how many urls didn't get scraped
print(len(bad_url))

1


In [42]:
# write out the urls that are left to scrape
with open('bad_url5.txt', 'wb') as f:
    pickle.dump(bad_url, f)

In [43]:
for row in bad_url:
    print(row)

https://www.tripadvisor.com/Restaurant_Review-g189541-d1418075-Reviews-La_Sirene-Copenhagen_Zealand.html
